In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.morphology import remove_small_holes, area_opening, skeletonize, area_closing, binary_erosion, binary_dilation
import pandas as pd
from matplotlib import rcParams
from skimage.measure import find_contours


In [2]:
COLOR = 'black'
plt.style.use("default")
rcParams["font.family"] = "serif"
rcParams["mathtext.fontset"] = "cm"
rcParams['figure.figsize'] = (28, 18)
rcParams['figure.titlesize'] = 20
rcParams['figure.dpi'] = 50
rcParams['axes.grid'] = True
rcParams['axes.facecolor'] = 'lightgray'
rcParams['legend.fontsize'] = 25
rcParams['axes.titlesize'] = 60
rcParams['axes.labelsize'] = 50
rcParams['ytick.labelsize'] = 20
rcParams['xtick.labelsize'] = 20
rcParams['text.color'] = COLOR
rcParams['text.color'] = COLOR
rcParams['axes.labelcolor'] = COLOR
rcParams['xtick.color'] = COLOR
rcParams['ytick.color'] = COLOR
rcParams['grid.color'] = COLOR
rcParams['text.latex.preamble'] = "\n".join([
    r'\usepackage[version=3]{mhchem}'
])

In [3]:
THRESHOLD = 120
MAX_PIXEL_VALUE = 255

### funciones

In [86]:

def cortar(frame, limites, PRUEBA=False, nombre='CORTE'):
    min_x, max_x, min_y, max_y = limites
    corte = frame[min_y:max_y, min_x:max_x]
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', corte)
    return corte

def gris(frame, PRUEBA=False, nombre='GRAYSCALE'):
    im_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_gray)
    return im_gray

def binarize(frame, th=THRESHOLD, PRUEBA=False, nombre='BINARY'):
    _, im_binary = cv.threshold(frame, th, MAX_PIXEL_VALUE, cv.THRESH_BINARY)
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_binary)
    return im_binary

def find(frame):
    y, x = np.where(frame==0)
    x_head = np.min(x)
    filas = np.unique(y)
    y_head = int(np.mean(filas))
    return x, y, x_head, y_head

def guillotina(frame, PRUEBA=False, nombre='DECAPITADO'):
    y, x = np.where(frame<=130)
    frame[y, x] = np.mean(frame)
    frame -= np.min(frame)
    frame = (frame/np.max(frame))*255
    
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', frame)
    return frame

def binarize_TAIL(frame, PRUEBA=False, nombre='BINARY_TAIL'):
    _, im_binary = cv.threshold(frame, THRESHOLD-15, MAX_PIXEL_VALUE, cv.THRESH_BINARY)
    arr = im_binary > 0
    im_binary = remove_small_holes(arr)*255

    # ESTO DE ACÁ COMENTADO ES LO QUE ACELERABA EL AREA OPENING:
    # no es el código más limpio del mundo pero funca, ahora no hacemos pero no lo borro ni corrijo todavía pq estoy trabajando cosas con eso
    y, x = np.where(im_binary==0)
    im_bin = im_binary[min(y)-1:max(y)+2, min(x)-1:max(x)+2]
    im_bin = area_opening(im_bin, 400)
    im_binary[min(y)-1:max(y)+2, min(x)-1:max(x)+2] = im_bin

    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_binary)
    return im_binary

def skeleton(frame, PRUEBA=False, nombre='ESQUELETO'):
    arr = frame < 255
    sk = skeletonize(arr)*255
    _, x = np.where(sk!=0)

    # esto cierra los globitos, es lo que faltaba antes:
    for c in np.unique(x):
        r = np.where(sk[:, c]!=0)[0]
        for p in r:
            sk[p, c] = 0
        sk[int(np.mean(r)), c] = 255
    tail = np.where(sk!=0)

    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', sk)
    return sk, tail

def track_TAIL(vs, limites, SHOW=False):
        # coordenadas de la cola en cada frame      
        frame_count = vs.get(7)
        pos_sk = []
        pos_tail = []
        i=0
        count=0
        lista_count=[]
        while(vs.isOpened()):
            ret, frame = vs.read()
            if ret:
                count +=1
                frame = cortar(frame, limites)

                # para que sólo tome cuando la cola está completa:
                im_izq = frame[:, 0:1]
                im_izq = cv.bitwise_not(binarize(gris(im_izq), th=100))
                im_der = frame[:, -2:-1]
                im_der = cv.bitwise_not(binarize(gris(im_der), th=160))
                if np.sum(im_izq) == 0:
                    if np.sum(im_der) == 0:
                        lista_count.append(count)
                        frame_bn = binarize_TAIL(guillotina(gris(frame)))
                        x_tail, y_tail = np.where(frame_bn==0)
                        sk, tail = skeleton(frame_bn)
                        pos_sk.append(sk)
                        pos_tail.append(tail)
                        if SHOW:
                            # frame[tail[0],tail[1]] = [0, 0, 255]
                            frame[x_tail,y_tail] = [0, 0, 255]                            
                            cv.imshow('frame', frame)
                        i+=1
                        if cv.waitKey(1) & 0xFF == ord('q'):
                            break
                else:
                    print('termina en:',count)
                    break
            else:
                print('Video no leído')
                break
        cv.destroyAllWindows()
        tiempo = [f for f in range(1, len(pos_sk)+1)]

        return i, lista_count, np.array(tiempo, dtype='object'), np.array(pos_sk, dtype='object'), np.array(pos_tail, dtype='object')

In [37]:
def tracker(vs, limites, SHOW=False):
        # Crea las listas vacias que van a ser nuestras mediciones
        pos_x = []
        pos_y = []
        
        frame_count = vs.get(7)
        tiempo = [f for f in range(1, int(frame_count))]
        i=0
        while(vs.isOpened()):
            i+=1
            ret, frame = vs.read()

            if ret:
                frame = cortar(frame, limites)
                frame_bn = binarize(gris(frame))
                mask = frame_bn>0
                frame_bn = remove_small_holes(mask,10)*255
                try:
                    x, y, x_head, y_head = find(frame_bn)
                    pos_x.append(x_head)
                    pos_y.append(y_head)
                    if SHOW:
                        frame[y, x] = [0, 255, 0]
                        frame = cv.circle(frame, (x_head, y_head), radius=1, color=(0, 0, 255), thickness=-1)
                        cv.imshow('frame', frame)
                except:
                    break
                if cv.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break
        cv.destroyAllWindows()
        # video.release()
        return np.array(tiempo), np.array(pos_x), np.array(pos_y)

def combineta(vs, limites, SHOW=False, SAVE=False):  # para generar el gif
        # coordenadas de la cola en cada frame
        tail_x = np.array([])
        tail_y = np.array([])        

        pos_x = []
        pos_y = []

        frame_count = vs.get(7)
        fps = 60
        tiempo = [f for f in range(1, int(frame_count))]

        # output = cv.VideoWriter('videos/video.mp4', cv.VideoWriter_fourcc(*"mp4v"), fps, frame_size)

        count = 0
        while(vs.isOpened()):
            ret, frame = vs.read()
            if ret:
                frame = cortar(frame, limites)

                # para que sólo tome cuando la cola está completa:
                im_der = frame[:, -2:-1]
                im_der = cv.bitwise_not(binarize(gris(im_der), th=170))

                # frame_width = int(frame.get(3))
                # frame_height = int(frame.get(4))
                # frame_size = (frame_width,frame_height)

                if np.sum(im_der) == 0:
                    count+=1
                    frame_bn_tail = binarize_TAIL(guillotina(gris(frame)))
                    _, x_tail, y_tail = skeleton(frame_bn_tail)
                    tail_x = np.append(tail_x, x_tail)
                    tail_y = np.append(tail_y, y_tail)

                    frame_bn = binarize(gris(frame))
                    mask = frame_bn>0
                    frame_bn = remove_small_holes(mask,10)*255

                    frame_width = limites[1]-limites[0]
                    frame_height = limites[2] - limites[3]
                    frame_size = (frame_width, frame_height)

                    try:
                        x, y, x_head, y_head = find(frame_bn)
                        pos_x.append(x_head)
                        pos_y.append(y_head)
                        if SHOW:
                            frame[y, x] = [0, 255, 0]
                            frame = cv.circle(frame, (x_head, y_head), radius=1, color=(0, 0, 0), thickness=-1)
                            frame[y_tail, x_tail] = [0, 0, 255]
                            # cv.imwrite(f'fotos\\analisis\\frame_{count}.jpg', frame)
                            cv.imshow('frame', frame)
                        # if SAVE:
                            # output.write(frame)
                            
                    except:
                        break
                    if cv.waitKey(1) & 0xFF == ord('q'):
                        break
                
            else:
                print('Video no abierto')
                break
        # output.release()
        cv.destroyAllWindows()

        return count, frame_size, tail_x, tail_y

In [77]:
# lim = [319, 950, 250, 505]
lim = [310, 950, 205, 505]

# el de 512 se arregló cambiando el th de im_izq a 100

In [27]:
im = cv.imread('Capturas\\Camo Snapshot 2024-05-17 12-04-24 - 10Hz.jpg')
im = cortar(im, lim)

In [87]:
video = cv.VideoCapture('videos\\ECO20-45mm-PECERA1\\ECO20-45mm-1,00Vpp-7,66Hz.mp4')
fc, lc, t, sk, tail = track_TAIL(video, lim, SHOW=True)
print(fc, len(lc), len(t), len(sk))

termina en: 1154
692 692 692 692


In [49]:
np.diff(lc)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [10]:
# video = cv.VideoCapture('videos\\ECO20-45mm-PECERA1\\ECO20-45mm-1,00V-3Hz.mp4')
video = cv.VideoCapture('videos\\ECO20-45mm-PECERA1\\ECO20-45mm-2,00Vpp-8Hz.mp4')
print(video.isOpened())
c, fs1, x, y = combineta(video, lim, SHOW=True)

# con el blur y el area opening a veces se pierde el final de alguna cola (500mVpp, 3Hz x ej)
# jugar con el orden de los filtros capaz

True


In [ ]:
print(np)

In [61]:
import os

for filename in os.listdir('videos\\ECO20-45mm-PECERA1'):
    print(f'Analizando {filename}')
    spl = filename.split('.')[0].split('-')
    v,f = spl[2],spl[3]
    video = cv.VideoCapture('videos\\ECO20-45mm-PECERA1\\'+filename)
    lim = [319, 950, 250, 505] #[xleft, xright, ytop, ydown]
    fc, lc, t, sk, tail = track_TAIL(video, lim, SHOW=True)
    print(fc, len(lc), len(t), len(sk))

Analizando ECO20-45mm-1,00V-3Hz.mp4
termina en: 2817
1634 1634 1634 1634
Analizando ECO20-45mm-1,00Vpp-10,01Hz.mp4
termina en: 689
340 340 340 340
Analizando ECO20-45mm-1,00Vpp-2Hz.mp4
termina en: 5028
3166 3166 3166 3166
Analizando ECO20-45mm-1,00Vpp-7,33Hz.mp4
termina en: 901
562 562 562 562
Analizando ECO20-45mm-1,00Vpp-7,66Hz.mp4
termina en: 879
417 417 417 417
Analizando ECO20-45mm-1,00Vpp-7Hz.mp4
termina en: 2074
1192 1192 1192 1192
Analizando ECO20-45mm-1,00Vpp-9,01Hz.mp4
termina en: 794
366 366 366 366
Analizando ECO20-45mm-1,01V-4Hz.mp4
termina en: 3133
1815 1815 1815 1815
Analizando ECO20-45mm-1,01Vpp-6Hz.mp4
termina en: 2513
1474 1474 1474 1474
Analizando ECO20-45mm-1,01Vpp-8Hz.mp4
termina en: 886
455 455 455 455
Analizando ECO20-45mm-1,02mVpp-5Hz.mp4
termina en: 2596
1564 1564 1564 1564
Analizando ECO20-45mm-1,49Vpp-8Hz.mp4
termina en: 571
263 263 263 263
Analizando ECO20-45mm-1,50Vpp-10,01Hz.mp4
termina en: 438
197 197 197 197
Analizando ECO20-45mm-1,50Vpp-2Hz.mp4
termina 